In [3]:
import importlib.util
import csv
import sys
import os
from unittest.mock import patch
from QiskitPBT.coordinator import Coordinator
from QiskitPBT.test_runner import TestRunner

PATH = os.path.abspath("")

def import_function(module_str, path, function_name):    
    spec = importlib.util.spec_from_file_location(module_str, path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_str] = module
    spec.loader.exec_module(module)    
    return getattr(module, function_name)

In [ ]:
def run_tests(algorithm_name, num_inputs, measurements):
    results = []
    for i in range(0, 2):
        circuit_function = import_function(f"{algorithm_name}_m{i}", f"{PATH}\\case_studies\\{algorithm_name}\\mutants\\{algorithm_name}_m{i}.py", algorithm_name)
        print(circuit_function())
        with (patch(f"QiskitPBT.case_studies.{algorithm_name}.{algorithm_name}.{algorithm_name}", circuit_function)):
            coordinator = Coordinator(num_inputs, 1)
            test_runner = TestRunner()
            result = coordinator.test(f"{PATH}\\case_studies\\{algorithm_name}", measurements) 

            # analyse results, add them to a list representing a csv file row
            num_circuits_executed = result.number_circuits_executed
            failed_properties = result.failed_property
            unique_properties = []
            failed_property_string = ""
            for property in failed_properties:
                if property.property.__class__ not in unique_properties:
                    failed_property_string += property.property.__class__.__name__ + " & "
                    unique_properties.append(property.property.__class__)
            failed_property_string = failed_property_string[:-3]
            num_unique_failed_properties = len(unique_properties)
            num_failed_properties = len(failed_properties)
            outcome = "Pass"
            if num_failed_properties > 0:
                outcome = "Fail"

            result_row = [f"{algorithm_name}_m{i}", str(outcome), str(num_circuits_executed), str(num_unique_failed_properties), str(num_failed_properties), str(failed_property_string)]

            results.append(result_row)
    # copy the for loop again for equivalent mutants, but change the name to f"{algorithm_name}_em{i}"
    for i in range(0, 2):
        circuit_function = import_function(f"{algorithm_name}_em{i}", f"{PATH}\\case_studies\\{algorithm_name}\\mutants\\{algorithm_name}_em{i}.py", algorithm_name)
        print(circuit_function())
        with (patch(f"QiskitPBT.case_studies.{algorithm_name}.{algorithm_name}.{algorithm_name}", circuit_function)):
            coordinator = Coordinator(num_inputs, 1)
            result = coordinator.test(f"{PATH}\\case_studies\\{algorithm_name}", measurements, run_optimization=True) 
            
            # analyse results, add them to a list representing a csv file row
            num_circuits_executed = result.number_circuits_executed
            failed_properties = result.failed_property
            unique_properties = []
            failed_property_string = ""
            for property in failed_properties:
                if property.property.__class__ not in unique_properties:
                    failed_property_string += property.property.__class__.__name__ + " & "
                    unique_properties.append(property.property.__class__)
            failed_property_string = failed_property_string[:-3]
            num_unique_failed_properties = len(unique_properties)
            num_failed_properties = len(failed_properties)
            outcome = "Pass"
            if num_failed_properties > 0:
                outcome = "Fail"
                
            result_row = [f"{algorithm_name}_m{i}", str(outcome), str(num_circuits_executed), str(num_unique_failed_properties), str(num_failed_properties), str(failed_property_string)]

            results.append(result_row)
            TestRunner.property_classes = []
            TestRunner.property_objects = []
            TestRunner.seeds_list_dict = {}
            TestRunner.num_inputs = 0
            TestRunner.do_shrinking = None
            TestRunner.max_attempts = 0
            TestRunner.num_measurements = 0
            TestRunner.test_execution_stats = None
    return results

test_and_store("quantum_teleportation")

number of inputs: 3, number of measurements: 2000
                    ┌───┐┌──────┐
q_0: ────────────■──┤ H ├┤0     ├
     ┌───┐     ┌─┴─┐└───┘│      │
q_1: ┤ H ├──■──┤ X ├──■──┤  Dcx ├
     └───┘┌─┴─┐└───┘┌─┴─┐│      │
q_2: ─────┤ X ├─────┤ X ├┤1     ├
          └───┘     └───┘└──────┘
{<class 'unitary_before_teleport_equal_unitary_after_teleport_property.UnitaryBeforeAndAfterTeleport'>, <class 'input_reg0_equal_to_output_reg2_property.Inq0EqualOutq2'>, <class 'not_teleported_registers_equal_to_plus_property.NotTeleportedPlus'>}
preflight steps 3.7744390964508057
circuit execution time 40.48219132423401
measurement allocation time 0.13564133644104004
p val calc time 0.16216063499450684
                    ┌───┐     
q_0: ────────────■──┤ H ├──■──
     ┌───┐     ┌─┴─┐└───┘  │  
q_1: ┤ H ├──■──┤ X ├──■────┼──
     └───┘┌─┴─┐└───┘┌─┴─┐┌─┴─┐
q_2: ─────┤ X ├─────┤ X ├┤ X ├
          └───┘     └───┘└───┘
{<class 'unitary_before_teleport_equal_unitary_after_teleport_property.UnitaryBeforeAnd

In [5]:
# I have written this specifically such that all the separate results of different configurations go in separate files 
# Just in case something goes horribly run when running, we won't lose everything
def test_and_store(algorithm_name):
    inputs = [3]
    shots = [2000]
    for input in inputs:
        for measurements in shots:
            print(f"number of inputs: {input}, number of measurements: {measurements}")
            filename = f"mutation_test_results/{algorithm_name}/{algorithm_name}_{input}_{measurements}_mt_results.csv"
            dir = os.path.dirname(filename)
            if dir and not os.path.exists(dir):
                os.makedirs(dir)
            with open(filename, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(["Mutant Name", "Result", "Number of Circuits Executed", "Number of Unique Failed Properties", "Number of Failed Properties", "Unique Failed Properties",])
                csvwriter.writerows(run_tests(algorithm_name, input, measurements))
                csvfile.close()

In [4]:
test_and_store("quantum_teleportation")

number of inputs: 3, number of measurements: 2000
                    ┌───┐┌──────┐
q_0: ────────────■──┤ H ├┤0     ├
     ┌───┐     ┌─┴─┐└───┘│      │
q_1: ┤ H ├──■──┤ X ├──■──┤  Dcx ├
     └───┘┌─┴─┐└───┘┌─┴─┐│      │
q_2: ─────┤ X ├─────┤ X ├┤1     ├
          └───┘     └───┘└──────┘
{<class 'input_reg0_equal_to_output_reg2_property.Inq0EqualOutq2'>, <class 'unitary_before_teleport_equal_unitary_after_teleport_property.UnitaryBeforeAndAfterTeleport'>, <class 'not_teleported_registers_equal_to_plus_property.NotTeleportedPlus'>}
preflight steps 3.788393259048462
circuit execution time 40.74892711639404
measurement allocation time 0.13530564308166504
p val calc time 0.15773701667785645
                    ┌───┐     
q_0: ────────────■──┤ H ├──■──
     ┌───┐     ┌─┴─┐└───┘  │  
q_1: ┤ H ├──■──┤ X ├──■────┼──
     └───┘┌─┴─┐└───┘┌─┴─┐┌─┴─┐
q_2: ─────┤ X ├─────┤ X ├┤ X ├
          └───┘     └───┘└───┘
{<class 'input_reg0_equal_to_output_reg2_property.Inq0EqualOutq2'>, <class 'unitary_befo

KeyboardInterrupt: 

In [5]:
# test_and_store("deutsch_jozsa")

In [6]:
# test_and_store("grovers_algorithm")

In [7]:
# test_and_store("quantum_fourier_transform")

In [8]:
# test_and_store("quantum_phase_estimation")

In [9]:
# algorithm_name = "quantum_teleportation"
# measurements = 2000
# num_inputs = 1
# coordinator = Coordinator(num_inputs, 1)
# result = coordinator.test(f"{PATH}\\case_studies\\{algorithm_name}", measurements) 
# print(result.failed_property)


In [10]:
# algorithm_name = "deutsch_jozsa"
# measurements = 2000
# num_inputs = 1
# coordinator = Coordinator(num_inputs, 1)
# result = coordinator.test(f"{PATH}\\case_studies\\{algorithm_name}", measurements) 
# print(result.failed_property)